## Airbnb project

In [131]:
import pandas as pd
import geopandas as gpd
import geoplot
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import seaborn as sns

In [132]:
hood = gpd.read_file('neighbourhoods.geojson')
metro = gpd.read_file('metro.geojson')
trains = gpd.read_file('comboios.geojson')
df = pd.read_csv('listings.csv')

# Metro dataset

In [133]:
df2 = metro.copy()
for i in range(len(df2)):
    if not '/' in df2['LINHA'][i]:
        df2.drop([i], axis=0, inplace=True)
df2 = df2.reset_index()

In [134]:
for i in range(len(metro)):
    if '/' in  metro['LINHA'][i]:
        metro['LINHA'][i] = metro['LINHA'][i].split('/')[0]

for j in range(len(df2)):
    df2['LINHA'][j] = df2['LINHA'][j].split('/')[1]
    
metro = pd.concat([metro, df2])

/var/folders/10/b37vvtzs1hx0zykkgsrfypbr0000gn/T/ipykernel_32579/857219963.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metro['LINHA'][i] = metro['LINHA'][i].split('/')[0]
/var/folders/10/b37vvtzs1hx0zykkgsrfypbr0000gn/T/ipykernel_32579/857219963.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['LINHA'][j] = df2['LINHA'][j].split('/')[1]


In [135]:
metro = pd.concat([metro, df2])

In [136]:
colors = {'Verde': 'green', 
          'Azul': 'blue', 
          'Azul/Verde': 'blue',
          'Amarela': 'red', 
          'Azul/Amarela': 'red',
          'Azul/Vermelha': 'blue',
          'Amarela/Vermelha': 'red',
          'Verde/Vermelha': 'green',
          'Amarela/Verde': 'red',
          'Vermelha': 'violet'
         }

In [137]:
metro['color_eng'] = (metro['LINHA']).map(colors)

In [138]:
yellow = metro[metro.LINHA == 'Amarela'].reset_index()
blue = metro[metro.LINHA == 'Azul'].reset_index()
green = metro[metro.LINHA == 'Verde'].reset_index()
red = metro[metro.LINHA == 'Vermelha'].reset_index()

In [139]:
#ax = geoplot.polyplot(hood, edgecolor='white', facecolor='lightgray', figsize=(12, 15))
#geoplot.pointplot(metro, ax=ax, hue='LINHA')
#gpd.GeoSeries([LineString(blue.geometry)]).plot(ax=ax)

# Metro Network

In [140]:
import networkx as nx
from pyvis.network import Network

In [141]:
metro_graph = pd.read_excel('metro.xlsx')

In [142]:
metro_graph = metro_graph.dropna()
metro_graph = metro_graph.drop(columns = ['NOME', 'LINHA', 'Unnamed: 0', 'OBJECTID'])

In [143]:
metro_graph.head()

,Origin,Destination
0,Reboleira,Amadora Este
1,Amadora Este,Alfornelos
2,Alfornelos,Pontinha
3,Pontinha,Carnide
4,Carnide,Colégio Militar


In [144]:
G=nx.Graph()
G=nx.from_pandas_edgelist(metro_graph, 'Origin', 'Destination')

In [145]:
net = Network(notebook=True)
net.from_nx(G)
net.width=500
net.height=500
net.show('metro.html')

In [146]:
metro_dict = dict(zip(metro.NOME, metro.geometry))

In [147]:
metro_graph['p1'] = (metro_graph['Origin']).map(metro_dict)
metro_graph['p2'] = (metro_graph['Destination']).map(metro_dict)

In [148]:
a = []
for i in range(len(metro_graph)):
    a.append((LineString([metro_graph.p1[i],metro_graph.p2[i]])))

metro_graph['LineString'] = a

In [149]:
metro_graph = gpd.GeoDataFrame(metro_graph)